# MNIST image classification

The MNIST data set is one of the most famous datasets in the machine learning community. It contains thousands of images of handwritten digits and is heavily used by machine learning beginners for building their first CNN.

## Your task
Create a neural network that can classify handwritten digits with an accuracy of over <b>98%</b>.

There is no need for writing code! All the code is prewritten, but in its current state, it would perform relatively poorly.  
Your job is to tweak the parameters until the CNN reaches the desired accuracy!

### NOTE
The cells should be executed one after the other. If you skip a cell, this can lead to errors down the line. Executing a cell multiple times is not a problem. If you go back to a previous cell, you also have to execute all the following cells.

After executing a cell with an `#auto` attribute manually once, it will execute automatically whenever you change a parameter.

Furthermore, be careful: Sometimes, certain combinations of parameters lead to errors!

## Visualization
Throughout the challenge, you can find code cells called 'VISUALIZE _'. If you execute these cells, you can see what is happening under the hood.  
Skipping them will not lead to errors.

## Testing
At the end of the notebook, you will find a testing environment that allows passing handwritten digits to the CNN. When trained properly, it should be able to tell which number you wrote down.

If you like, you can train the CNN in its current state to see how badly it performs. This will make the learning process even more impressive.

## Tips
The next few cells are tips. You can use them one after the other when you are stuck. Skipping them will not lead to errors.

In [ ]:
#@title Tip 1
print(
"Increasing parameters to their max won't always increase performance. \
In many cases, big parameters even smaller the performance. \
So bigger is not always better!"
)

In [ ]:
#@title Tip 2
print(
"For the output layer, you need the softmax activation. \
This activation function converts all the values into probabilities."
)

In [ ]:
#@title Tip 3
print(
"For all other layers, relu is a great idea. \
This function will return 0 for all negative x, and x for all positive x."
)

In [ ]:
#@title Tip 4
print("For multi-class classification, categorical_cross_entropy is a fantastic loss function.")

In [ ]:
#@title Tip 5
print("A big batch size often ruins your result. Try something between 1000 and 2000")

In [ ]:
#@title Tip 6
print(
"Using only valid padding results in most of the image data getting lost.\n\
Using only same padding results in the CNN not being able to extract essential features.\n\
Try using same for the conv-layers and valid for the max-pooling-layers."
)

In [ ]:
#@title Tip 7
print(
"Consider using about one thousand neurons per normal layer.\n\
Also, consider 2 to 3 normal layers in total.\n\
Too many or to view neurons or layers will lead to bad results."
)

In [ ]:
#@title import dependencies

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from cv2 import resize, INTER_CUBIC
from PIL import Image as PILImage
from io import BytesIO

In [ ]:
#@title load & prepare images & labels
(x_train, y_train), (x_test, y_test) = load_data()

## prepare images

# images are currently represented as values between 0 and 255
# for a NN, it's best to have them between 0 and 1
x_train = x_train/255
x_test = x_test/255

# changes the dimension, so the NN can better work with it
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## prepare labels

# labels are currently represented as a digit between 0 and 9
# for training the NN, it's best to represent them as an array of probabilities
# so 1 wold become [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_cat_train = to_categorical(y_train,10)
y_cat_test = to_categorical(y_test,10)

In [ ]:
#@title VISUALIZE image `#auto` {run: "auto"}
image =  0#@param

if image < 0:
  print("Sorry, there is no image before index 0")
elif image >= len(x_train):
  print(
      f"Ups! There are only {len(x_train)} images. \
      Choose an image between 0 and {len(x_train)-1}"
  )
else:
  plt.imshow(x_train[image].reshape((28, 28)), cmap='gray')

In [ ]:
#@title # create CNN `#auto` {run: "auto"}

### PARAMETERS

#@markdown ---
#@markdown ### Convolutional layers
conv_layers = 0 #@param {type: 'slider', min: 0, max: 5}
padding = 'valid' #@param ["valid", "same"]
pooling_padding = 'valid' #@param ["valid", "same"]
kernel_size = 1 #@param {type: "slider",  min: 1, max: 5}
filters = 1 #@param {type: "slider", min: 1, max: 10}
conv_activaton_function = 'linear' #@param ["elu", "exponential", "linear", "relu", "sigmoid", "softmax"]

#@markdown ---
#@markdown ### Normal layers
normal_layers = 0 #@param {type: 'slider', min: 0, max: 5}
neurons_per_layer = 1 #@param {type: 'slider', min: 1, max: 2000}
normal_activaton_function = 'linear' #@param ["elu", "exponential", "linear", "relu", "sigmoid", "softmax"]

#@markdown ---
#@markdown ### Output layer
output_activation_function = 'linear' #@param ["elu", "exponential", "linear", "relu", "sigmoid", "softmax"]

#@markdown ---
#@markdown ### CNN settings
optimizer = 'sgd' #@param ["adam", "rmsprop", "sgd"]
loss_function = 'poisson' #@param ["binary_crossentropy", "categorical_crossentropy", "mean_squared_error", "poisson"]

### CODE

# create the empty model and add an Input-layer
model = Sequential(name="TUMKolleg_CNN")
model.add(Input((28, 28, 1)))

# add the convolutional and the max-pooling layers
for i in range(0, conv_layers):
  model.add(Conv2D(filters=filters, activation=conv_activaton_function, kernel_size=(kernel_size, kernel_size), padding=padding, name=f"conv_{i}"))
  model.add(MaxPool2D(padding=pooling_padding, name=f"maxpooling_{i}"))

# flatten the output of the convolutional layers, so it can be processed by 
# the normal layers
model.add(Flatten(name="flattening_layer"))

# add the normal layers
for i in range(0, normal_layers):
  model.add(Dense(neurons_per_layer, activation=normal_activaton_function, name=f"dense_{i}"))

# the output layer needs a special activation function, so it
# outputs a probability for each possible prediction
model.add(Dense(10, activation=output_activation_function, name="output_layer"))

# finally, compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"])

In [ ]:
#@title VISUALIZE CNN layers
model.summary()

In [ ]:
#@title train the CNN

#@markdown executing this cell mutiple times without creating a new CNN, will
#@markdown train the same network again. (i.e. 2 x 10 epoch is like 20 epochs
#@markdown at once)

epochs = 2 #@param {type: 'slider', min: 0, max: 20}
batch_size = 1000 #@param {type: 'slider', min: 1000, max: 5000}
result_html = True #@param {type: "boolean"}

success_html = """
      <HTML>
        <style>
          body { background-color: green }
          h1 { color: white }
        </style>
        <h1>Congratulations! You successfully achieved <b>%s</b> accuracy!</h1>
      </HTML>
"""
failure_html = """
      <HTML>
        <style>
          body { background-color: red }
          h1 { color: white }
        </style>
        <h1>Ups! You just achieved <b>%s</b> accuracy!</h1>
      </HTML>
"""

model.fit(x_train, y_cat_train, epochs=epochs, batch_size=batch_size,validation_data=(x_test,y_cat_test))
stats = pd.DataFrame(model.history.history)
accuracy = model.evaluate(x_test,y_cat_test,verbose=0)[1]

if accuracy > 0.98:
  if result_html:
    display(HTML(success_html % f"{round(accuracy*100, 2)}%"))
  else:
    print(f"succeded with accuracy: {round(accuracy*100, 2)}%")
else:
  if result_html:
    display(HTML(failure_html % f"{round(accuracy*100, 2)}%"))
  else:
    print(f"failed with accuracy: {round(accuracy*100, 2)}%")

In [ ]:
#@title VISUALIZE training results
stats[['accuracy','val_accuracy']].plot()
stats[['loss','val_loss']].plot()

In [ ]:
#@title VISUALIZE prediction results

predictions = model.predict_classes(x_test)
print(classification_report(y_test,predictions))
plt.figure(figsize=(15,10))
sns.heatmap(confusion_matrix(y_test,predictions),annot=True)

In [ ]:
#@title # Initialize Testing suite

canvas_html = """
<canvas width=%d height=%d, style="border: 5px solid red;"></canvas>
<br>
<button>Finish</button>
<p id="result"></p>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(w=400, h=400, line_width=25):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  return binary

def to_gray(image):
  new = np.zeros((28, 28, 1))
  for x in range(0, 28):
    for y in range(0, 28):
      new[x, y] = image[x, y, 3]/255
  return new

In [ ]:
#@title # Test hand written digits

image = draw()

image = PILImage.open(BytesIO(image))
image = np.array(image)
image = resize(image, dsize=(28, 28), interpolation=INTER_CUBIC)
image = to_gray(image)
image = image.reshape((1, 28, 28, 1))

plt.imshow(image.reshape((28, 28)), cmap='gray')

res = model.predict_classes(image)
print(res)